## **Lernziele**

Durch das Bearbeiten dieser Übungen werden Sie:

- Retrieval-Augmented Generation (RAG) und seine Komponenten verstehen.
- PDF-Dokumente effektiv laden, vorverarbeiten und verarbeiten.
- Textdaten in Embeddings für effiziente Suche umwandeln.
- Dokumenten-Retrieval-Systeme mit LangChain und FAISS implementieren und testen.
- Retrieval-Systeme mit kostenlosen Sprachmodellen (LLMs) von ChatGroq integrieren.
- Ein interaktives chatbasiertes Frage-Antwort-System aufbauen.

---

## **Übung 1: Setup und Aufwärmen**

In dieser Übung richten Sie Ihre Umgebung ein und wählen ein geeignetes Sprachmodell aus.

**Schritte:**

1. **Umgebungsvariablen laden:** Stellen Sie sicher, dass Ihre Umgebungsvariablen (z.B. API-Schlüssel, Tokens) sicher gespeichert und geladen werden.
2. **LLM auswählen:** Wählen Sie ein kostenloses LLM-Modell von ChatGroq aus.
3. **Modell instanziieren:** Erstellen Sie eine Instanz Ihres gewählten Modells.

In [1]:
# Import necessary libraries
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [2]:
# warnings.filterwarnings("ignore")
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.5,
    max_tokens=1024,
    timeout=None,
    max_retries=2
)

---

## **Übung 2: Datenaufnahme**

In dieser Übung lernen Sie, PDF-Daten in eine Python-Umgebung zu laden.

**Schritte:**

1. **PDF-Loader importieren:** Verwenden Sie LangChains `PyPDFLoader`.
2. **PDF-Datei laden:** Erstellen Sie eine Funktion zum Lesen der PDF-Datei.
3. **PDF-Inhalt anzeigen:** Geben Sie die Seitenanzahl und den Inhalt der ersten Seite aus.

In [3]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from pathlib import Path

data_path = Path("data") 

loader = DirectoryLoader(
    str(data_path),
    glob="**/*.md",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"},
)

docs = loader.load()

print(f"{len(docs)} Dokumente geladen")


27 Dokumente geladen


---

## **Übung 3: Dokument-Chunking**

Diese Übung führt das Aufteilen großer Dokumente in handhabbare Textblöcke ein.

**Schritte:**

1. **Text-Splitter importieren:** Verwenden Sie `RecursiveCharacterTextSplitter`.
2. **Dokument aufteilen:** Schreiben Sie eine Funktion, die geladene Dokumente in Chunks aufteilt.
3. **Funktion testen:** Überprüfen Sie das Ergebnis, indem Sie die resultierenden Chunks anzeigen.

In [9]:
# Import RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunk_documents(documents, chunk_size=200, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents=documents)
    
    # Just to add id for etch chunks to map it later 
    for i, chunk in enumerate(chunks):
         chunk.metadata.update({
        "id": f"chunk_{i}",
    })
    
    return chunks


react_chunks = chunk_documents(docs)

In [10]:
# Execute your chunking function and display results here
len(react_chunks)

1115


---

## **Übung 4: Embedding und Speicherung**

In dieser Übung erstellen Sie Embeddings aus Textblöcken und speichern sie effizient.

**Schritte:**

1. **Embedding-Modell wählen:** Verwenden Sie `sentence-transformers/all-mpnet-base-v2` von Hugging Face.
2. **Embeddings generieren:** Wandeln Sie Dokument-Chunks in Embeddings um.
3. **Embeddings speichern:** Speichern Sie diese Embeddings lokal mit FAISS.

In [19]:
# Import libraries
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.faiss import DistanceStrategy

def embed_and_store(chunks, db_name):
    """
    This function uses the open-source embedding model HuggingFaceEmbeddings 
    to create embeddings and store those in a VectorStore called FAISS, 
    which allows for efficient similarity search
    """
    # instantiate embedding model
    embedding = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2',
        encode_kwargs={"normalize_embeddings": True}
    )
    # create the vector store 
    vectorstore = FAISS.from_documents(
        documents=chunks,
        embedding=embedding,
        distance_strategy=DistanceStrategy.COSINE  # or DistanceStrategy.DOT or DistanceStrategy.L2 
        
    )
    # save VectorStore locally
    vectorstore.save_local(f"../vector_databases/vector_db_{db_name}")
    return vectorstore



In [20]:
# Generate embeddings and save them locally
all_embedding=embed_and_store(chunks=react_chunks, db_name="ezb")

---

## **Übung 5: Retrieval aus FAISS**

Hier lernen Sie, wie man Dokumente aus einer Vektordatenbank mithilfe von Embeddings abruft.

**Schritte:**

1. **Embeddings laden:** Laden Sie gespeicherte Embeddings aus der FAISS-Datenbank.
2. **Retrieval implementieren:** Erstellen Sie eine Logik zum Abrufen relevanter Chunks basierend auf Abfragen.
3. **Retriever testen:** Führen Sie die Suche mit Beispielabfragen durch.

In [21]:
# Implement retrieval logic from your FAISS database
def retrieve_from_vector_db(vector_db_path):
    """
    this function splits out a retriever object from a local VectorStore
    """
    # instantiate embedding model
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2',
        encode_kwargs={"normalize_embeddings": True}
    )
    react_vectorstore = FAISS.load_local(
        folder_path=vector_db_path,
        embeddings=embeddings,
        allow_dangerous_deserialization=True,
        distance_strategy=DistanceStrategy.COSINE  # or DistanceStrategy.DOT or DistanceStrategy.L2 
    )
    retriever = react_vectorstore.as_retriever()
    return retriever ,react_vectorstore

# Load the retriever and index
react_retriever,react_vectorstore = retrieve_from_vector_db("../vector_databases/vector_db_ezb")
type(react_retriever),type(react_vectorstore)

'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 36c00454-a6c7-496f-a3e4-ec76fff61c4c)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-mpnet-base-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


(langchain_core.vectorstores.base.VectorStoreRetriever,
 langchain_community.vectorstores.faiss.FAISS)

In [22]:
# Test your retrieval system with queries
react_retriever.get_relevant_documents("Inflation auf mittlere Sicht",k=3)

C:\Users\stf\AppData\Local\Temp\ipykernel_10236\2424041750.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  react_retriever.get_relevant_documents("Inflation auf mittlere Sicht",k=3)


[Document(id='ce96cb42-d707-4025-bbd1-0d2b4cd738d2', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2026-02-18T16:13:13+01:00', 'author': 'Europäische Zentralbank', 'moddate': '2026-02-19T08:48:21+01:00', 'title': 'Wirtschaftsbericht, Ausgabe 1 / 2026', 'source': '../documents/2026-01-ezb-wb-data.pdf', 'total_pages': 135, 'page': 3, 'page_label': '4', 'id': 'chunk_24'}, page_content='bestätigte erneut, dass sich die Inflation auf mittlere Sicht bei seinem Zielwert von \n2 % stabilisieren dürfte. Die Wirtschaft zeigt sich in einem schwierigen globalen'),
 Document(id='d45b1616-5562-45ac-a163-7999faf4d4ea', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2026-02-18T16:13:13+01:00', 'author': 'Europäische Zentralbank', 'moddate': '2026-02-19T08:48:21+01:00', 'title': 'Wirtschaftsbericht, Ausgabe 1 / 2026', 'source': '../documents/2026-01-ezb-wb-data.pdf', 'total_pages': 135, 'page': 6, 'page_label': '7', 'id': 'chunk_84'}, page_content='sorgen, das

---

## **Übung 6: Retrieval mit LLM verbinden**

Nun verbinden Sie das Dokument-Retrieval mit dem Sprachmodell.

**Schritte:**

1. **Retrieval-Chain erstellen:** Verknüpfen Sie Ihr Retrieval-System mit Ihrem instanziierten LLM.
2. **Chain testen:** Bestätigen Sie die Funktionalität, indem Sie Antworten aus abgerufenen Dokumenten generieren.

In [24]:
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

# Write a function to create retrieval and document processing chains
def connect_chains(retriever):
    """
    this function connects stuff_documents_chain with retrieval_chain
    """
    stuff_documents_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=hub.pull("langchain-ai/retrieval-qa-chat")
    )
    retrieval_chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=stuff_documents_chain
    )
    return retrieval_chain


react_retrieval_chain = connect_chains(react_retriever)

In [27]:
# Invoke your chain with a sample question
output = react_retrieval_chain.invoke(
    {"input": "Bei welchem Wert will die EZB die Inflation mittelfristig sehen?"}
)
output['answer']

'Die EZB will die Inflation mittelfristig bei einem Wert von 2 % stabilisieren.'

In [30]:
react_retrieval_chain.invoke(
    {"input": "Bitte fasse die Konjunkturentwicklung nach der vorläufigen Schnellschätzung von Eurostat in 5 Sätzen zusammen."}
)['answer']

'Die vorläufige Schnellschätzung von Eurostat weist ein Wirtschaftswachstum von 0,3 % im vierten Quartal 2025 aus. Dieses Wachstum ist hauptsächlich auf die Entwicklung im Euroraum zurückzuführen. Die genauen Details über die Ursachen des Wachstums sind nicht im bereitgestellten Text erwähnt. Die Wirtschaftsentwicklung wird durch verschiedene Indikatoren wie den Arbeitsplatzverlust und den Preisdruck auf der Wertschöpfungskette beurteilt. Die genauen Zahlen und Details zu diesen Indikatoren sind jedoch nicht im bereitgestellten Text enthalten.'

---

## **Übung 7: Interaktives Chat-System**

In der letzten Übung bauen Sie ein interaktives chatbasiertes Abfragesystem.

**Schritte:**

1. **Chat-Oberfläche erstellen:** Entwickeln Sie eine einfache Funktion für interaktive Abfragen.
2. **Chat ausführen:** Ermöglichen Sie es Nutzern, Fragen zu stellen und sofortige Antworten zu erhalten.

In [34]:
# Define your interactive chat querying function

while True:
    # 1. Do something (runs at least once)
    user_query = input("Frag ans RAG-System: ")
    
    # 2. Check the "until" condition
    if user_query == 'quit':
        break
    print(react_retrieval_chain.invoke({"input":user_query})['answer'])
    print("...")

Ja, das weltweite Wirtschaftswachstum hat sich insgesamt weiterhin als robust erwiesen. Dies ist zu verdanken an:

1. Kräftiges Wirtschaftswachstum im dritten Quartal 2025
2. Robuste Nachfrage nach höher rentierenden Staatsanleihen

Diese Faktoren haben dazu beigetragen, dass das weltweite Wirtschaftswachstum stabil und kräftig bleibt.
...
Das Dokument sagt, dass der Einkaufsmanagerindex (EMI) für die Produktion im verarbeitenden Gewerbe und im Dienstleistungssektor im Januar 2026 weitgehend stabil geblieben ist.
...
Es scheint, dass der bereitgestellte Text nicht explizit fünf Kernaussagen enthält. Der Text scheint eher ein Fragment zu sein, das verschiedene Ideen und Aussagen enthält, aber keine klaren Kernaussagen. Es gibt jedoch einige Schlüsselaussagen, die ich herausgreifen kann:

1. Die Fähigkeiten, Kenntnisse und Kompetenzen für Aktivitäten mit Brennstoffen sind nicht ein separates Spektrum an Fähigkeiten.
2. Brennstoffe mit höheren Grenzkosten führen tendenziell zu höheren Str

In [26]:
# Run and test your interactive chat system

---

## **Fazit & Reflexion**

Nach Abschluss dieser Übungen:

- Fassen Sie die gelernten Schlüsselkonzepte zusammen.
- Reflektieren Sie über die Wirksamkeit und Einschränkungen des kostenlosen LLM und des von Ihnen aufgebauten RAG-Systems.
- Überlegen Sie, wie Sie Ihr System in praktischen Anwendungen verbessern oder erweitern könnten.

---